In [20]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [30]:
torch.manual_seed(0)
device = torch.device("".join(["cuda:",f'{CUDA_ID}']) if torch.cuda.is_available() else "cpu")

state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

CUDA_ID = 1
dt = .2
T = 1000 #Run simulation for 1000 hours.
N = int(T / dt) 
T_span = np.linspace(0, T, N + 1)
T_span_tensor = torch.reshape(torch.Tensor(T_span), [1, N + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
LR = 1e-3
niter = 4000
pretrain_iter = 2000
batch_size = 1 #Number of sets of observation outputs to sample per set of parameters.

In [31]:
temp_ref = 283

#System parameters from deterministic model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_CO2 = 0.0001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [32]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
I_S_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * T_span_tensor) #Exogenous SOC input function
I_D_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * T_span_tensor) #Exogenous DOC input function

In [33]:
#Read-in deterministic data observations for use in inference.
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', 4, 1000, 0.1)
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', 5, 1000, 0.1)
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', 5, 1000, 0.1)

In [34]:
def neg_log_lik(C_path, T_span_tensor, dt, I_S_tensor, I_D_tensor, drift_diffusion, params_dict, temp_ref):
    drift, diffusion_sqrt = drift_diffusion(C_path[:, :-1, :], T_span_tensor[:, :-1, :], I_S_tensor[:, :-1, :], I_D_tensor[:, :-1, :], params_dict, temp_ref)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_path[:, :-1, :] + drift * dt, scale_tril = diffusion_sqrt * math.sqrt(dt))
    return -euler_maruyama_state_sample_object.log_prob(C_path[:, 1:, :]).sum(-1)

In [35]:
obs_model_CON_noCO2 = ObsModel(device = device, times = obs_times, dt = dt, mu = obs_means_CON[:-1, :], scale = obs_error_CON[:, :-1])
obs_model_AWB_noCO2 = ObsModel(device = device, times = obs_times, dt = dt, mu = obs_means_AWB[:-1, :], scale = obs_error_AWB[:, :-1])
obs_model_AWB_ECA_noCO2 = ObsModel(device = device, times = obs_times, dt = dt, mu = obs_means_AWB_ECA[:-1, :], scale = obs_error_AWB_ECA[:, :-1])

In [36]:
def train(net, niter, pretrain_iter, batch_size, obs_model, T_span_tensor, I_S_tensor, I_D_tensor, drift_diffusion, params_dict, analytical_steady_state_init):
    if pretrain_iter >= niter:
        raise Exception("pretrain_inter must be < niter.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = analytical_steady_state_init(I_S_tensor[0, 0, 0].item(), I_D_tensor[0, 0, 0].item(), params_dict) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(batch_size, 1, 1).to(device) #Assign initial conditions to C_path.
    with tqdm(total = niter, desc = f'Train Diffusion', position = -1) as t:
        for iter in range(niter):
            net.train()
            optimizer.zero_grad()
            C_path, log_prob = net() #Obtain paths with solutions at times after t0.
            C_path = torch.cat([C0, C_path], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= pretrain_iter:
                l1_norm_element = C_path - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_path - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_path mean =', C_path.mean(-2))
                    print('\nC_path =', C_path)
                l1_norm.backward()
            else:
                log_lik = neg_log_lik(C_path, T_span_tensor.to(device), dt, I_S_tensor.to(device), I_D_tensor.to(device), drift_diffusion, params_dict, temp_ref)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model(C_path)
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_path mean =', C_path.mean(-2))
                    print('\n C_path =', C_path)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            t.update()

In [37]:
net_SCON_c = SDEFlow(device = device, obs_model = obs_model_CON_noCO2, state_dim = state_dim_SCON, T = T, dt = dt, N = N, batch_size = batch_size).to(device)
optimizer = optim.Adam(net_SCON_c.parameters(), lr = LR)

In [ ]:
train(net_SCON_c, niter, pretrain_iter, batch_size, obs_model_CON_noCO2, T_span_tensor, I_S_tensor, I_D_tensor, drift_diffusion_scon_c, SCON_C_params_dict, analytical_steady_state_init_con)


Train Diffusion:   0%|          | 0/4000 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000022788.976562. Best norm loss value is: 227889.765625.

C_path mean = tensor([[0.6117, 0.6914, 0.9139]], grad_fn=<MeanBackward1>)

C_path = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [8.2709e-01, 7.9033e-01, 8.8175e-01],
         [5.8314e-01, 6.9806e-01, 7.4934e-01],
         ...,
         [8.9601e-01, 7.9831e-01, 8.0336e-01],
         [8.9254e-01, 7.0059e-01, 2.4620e-01],
         [9.2251e-01, 5.9186e-01, 1.1602e-02]]], grad_fn=<CatBackward>)
